## ResNet18的测试代码

In [33]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn as nn
import torch

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)  # 修改这里，将x改为out
        out = self.bn2(out)
        out += self.downsample(identity)
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def ResNet18(num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

def process_data(data_dir):
    """
    原始数据处理
    param:
        data_dir: str, 数据集文件夹路径
    return:
        dataloader: DataLoader, 数据加载器
        class_names: list, 类别名称列表
    """
    batch_size=20
    transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    X = DataLoader(dataset, batch_size=20, shuffle=False)

    labels = [label for _, label in dataset] # 一个一个取图片滴label
    return X, labels





def predict(X):
    """
    模型预测
    param:
        model: nn.Module, 训练好的模型
        dataloader: DataLoader, 数据加载器
    return:
        y_predict: np.ndarray, 数据预测标签
    """

    device='cpu'
    all_predictions = []
    with torch.no_grad():  # 禁用梯度计算，节省内存和计算资源
        for data, target in X:
            # 将数据传递给模型进行推理
            output = model(data)

            # 假设模型输出是 logits，我们使用 softmax 获取概率
            probabilities = torch.softmax(output, dim=1)

            # 获取预测结果（最大概率的类别索引）
            predictions = torch.argmax(probabilities, dim=1)
            # print( predictions )

            
            # 将预测结果转换为 NumPy 数组并保存
            all_predictions.extend(predictions.cpu().numpy())
            
    y_predict = all_predictions
    return y_predict

data_dir = './dataset/modify'  # 替换为实际数据集路径
# 加载模型和权重
num_classes = 376  # 替换为您的分类数
model = ResNet18(num_classes=num_classes)
model.load_state_dict(torch.load('./ResNet.pt', map_location = torch.device('cpu') ))
model.eval()  # 设置模型为评估模式
dataloader, data_class_names = process_data(data_dir) 

In [34]:
# 群友写的类std后端
if __name__ == "__main__":
    X , labels = process_data(data_dir)
    y_predict = predict(X)
    # print( "labels:\n",labels )
    # print( "y_predict\n",y_predict )
    print(1111111111111)
    correct = 0
    total = len( labels )
    for true_label , pred_label in zip( labels , y_predict ):
        # print( f"测试中：{true_label}-----{pre_label}" )
        if true_label == pred_label:
            correct += 1
    accuracy = correct / total
    print(f"Accuarcy:{accuracy:.10%}")
        

1111111111111
Accuarcy:99.4635816242%


In [ ]:
## GoogleNet的测试代码

In [3]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn as nn
import torch


class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3_reduce, ch3x3, ch5x5_reduce, ch5x5, pool_proj):
        super(Inception, self).__init__()
        self.branch1 = nn.Conv2d(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3_reduce, kernel_size=1),
            nn.Conv2d(ch3x3_reduce, ch3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5_reduce, kernel_size=1),
            nn.Conv2d(ch5x5_reduce, ch5x5, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)


class GoogleNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(GoogleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4) 
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)  
        x = self.fc(x)
        return x

def process_data(data_dir):
   
    batch_size=20
    transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    X = DataLoader(dataset, batch_size=20, shuffle=False)

    labels = [label for _, label in dataset] # 一个一个取图片滴label
    return X, labels





def predict(X):

    device='cpu'
    all_predictions = []
    with torch.no_grad():  
        for data, target in X:
            
            output = model(data)
            
            probabilities = torch.softmax(output, dim=1)

            predictions = torch.argmax(probabilities, dim=1)
            # print( predictions )

            
            all_predictions.extend(predictions.cpu().numpy())
            
    y_predict = all_predictions
    return y_predict


num_classes = 376  
model = GoogleNet(num_classes=num_classes)
model.load_state_dict(torch.load('./GoogleNet.pt', map_location = torch.device('cpu') ))
model.eval()  

GoogleNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (inception3a): Inception(
    (branch1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (branch2): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (branch3): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (branch4): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (1): Conv2d(192, 32, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (inception3b): Inception(
    (

In [5]:
print("process")
# 群友写的类std后端
if __name__ == "__main__":
    data_dir = './dataset/train'  # 替换为实际数据集路径
    print(11111111111111111111111111111111111)
    X , labels = process_data(data_dir)
    y_predict = predict(X)
    # print( "labels:\n",labels )
    # print( "y_predict\n",y_predict )
    print(2222222222222222222222222222222222222)
    correct = 0
    total = len( labels )
    for true_label , pred_label in zip( labels , y_predict ):
        # print( f"测试中：{true_label}-----{pre_label}" )
        if true_label == pred_label:
            correct += 1
    accuracy = correct / total
    print(f"Accuarcy:{accuracy:.10%}")
        

process
11111111111111111111111111111111111
2222222222222222222222222222222222222
Accuarcy:88.0665931713%
